In [1]:
import pandas as pd
import numpy as np
import time
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoremiscv2
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoreusagev2

In [2]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['GAME_ID'])
    # Filter out any row that is joined to itself.
    
    for columnName in joined:
        if(columnName == 'TEAM_ID_A_x'):
            joined = joined.rename(columns = {'TEAM_ID_A_x':'TEAM_ID_A'})
        
    for columnName in joined:
        if(columnName == 'TEAM_ID_B_x'):
            joined = joined.rename(columns = {'TEAM_ID_B_x':'TEAM_ID_B'})  
        
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [3]:
nba_teams = ['Cleveland Cavaliers','Boston Celtics','Golden State Warriors','Houston Rockets','Utah Jazz'
             ,'Philadelphia 76ers','Toronto Raptors','New Orleans Pelicans','Washington Wizards','Miami Heat'
            ,'Milwaukee Bucks','Indiana Pacers','Oklahoma City Thunder','San Antonio Spurs','Portland Trail Blazers',
             'Minnesota Timberwolves','Chicago Bulls','Dallas Mavericks','Sacramento Kings','Los Angeles Lakers'
            ,'Orlando Magic','Denver Nuggets','LA Clippers','New York Knicks','Memphis Grizzlies','Detroit Pistons'
            ,'Charlotte Hornets','Atlanta Hawks','Phoenix Suns','Brooklyn Nets']

In [4]:
def remove_unnamed(frame):
    for columnName in frame:
        if(columnName[0:7] == 'Unnamed'):
            frame = frame.drop(columns = [columnName])
    return frame

In [5]:
def pull_reg_season(start_year, start_date, end_date):
    games_base = leaguegamefinder.LeagueGameFinder()
    games = games_base.get_data_frames()[0]
    games_xxxx = games[games.SEASON_ID.str[-4:] == start_year]
    games_xxxx = games_xxxx[games_xxxx.TEAM_NAME.isin(nba_teams)]
    regular_xxxx = games_xxxx[(games_xxxx.GAME_DATE > start_date) & (games_xxxx.GAME_DATE < end_date)]
    regular_merged = combine_team_games(regular_xxxx,keep_method = None)
    return regular_merged

In [6]:
def pull_stats(api, index, reg_season):
    box = pd.DataFrame()
    ids = reg_season['GAME_ID']
    subset1 = ids[0:308]
    subset2 = ids[308:616]
    subset3 = ids[616:924]
    subset4 = ids[924:1231]
    subset_list = [subset1, subset2, subset3, subset4]
    for i in subset_list:
        for j in i:
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
        time.sleep(60)
    return box
        

            
                
            
    

In [29]:
def pull_stats_super_slow(api, index, reg_season):
    box = pd.DataFrame()
    ids = reg_season['GAME_ID']
    subset1 = ids[0:308]
    subset2 = ids[308:616]
    subset3 = ids[616:924]
    subset4 = ids[924:1231]
    subset_list = [subset1, subset2, subset3, subset4]
    for i in subset_list:
        for j in i:
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
            time.sleep(20)
        
    return box

In [7]:
def pull_stats_slow(api, index, reg_season):
    box = pd.DataFrame()
    ids = reg_season['GAME_ID']
    subset1 = ids[0:77]
    subset2 = ids[77:154]
    subset3 = ids[154:231]
    subset4 = ids[231:308]
    subset5 = ids[308:385]
    subset6 = ids[385:462]
    subset7 = ids[462:539]
    subset8 = ids[539:616]
    subset9 = ids[616:693]
    subset10 = ids[693:770]
    subset11 = ids[770:847]
    subset12 = ids[847:924]
    subset13= ids[924:1001]
    subset14 = ids[1001:1078]
    subset15 = ids[1078:1155]
    subset16 = ids[1155:1231]
    subset_list = [subset1, subset2, subset3, subset4,subset5, subset6, subset7, subset8,
                  subset9, subset10, subset11, subset12,subset13, subset14, subset15, subset16]
    counter = 0
    for i in subset_list:
        for j in i:
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
            print(counter)
            counter = counter + 1
        time.sleep(300)
    return box

In [8]:
def reg_clean(reg):
    reg = reg.drop(columns = ['GAME_DATE_A', 'MATCHUP_A', 'WL_A', 'MIN_A','SEASON_ID_A'
                              , 'TEAM_ABBREVIATION_A', 'TEAM_NAME_A'])
    
    return reg
    
    

In [9]:
def advancedboxscore_clean(advanced):
    advanced = advanced.drop(columns = ['TEAM_ID_B','TEAM_ABBREVIATION_A','TEAM_ABBREVIATION_B'
                                   ,'TEAM_CITY_A','TEAM_CITY_B','TEAM_NAME_A','TEAM_NAME_B','MIN_A'])
    
    return advanced
    


In [10]:
def traditional_clean(traditional):
    bench = traditional.drop_duplicates(subset = ['GAME_ID','TEAM_NAME_A'], keep = 'last')
    starters = traditional.drop_duplicates(subset = ['GAME_ID','TEAM_NAME_A'], keep = 'first')
    bench = bench.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'STARTERS_BENCH_A', 'MIN_A','TEAM_ID_B', 'TEAM_NAME_B',
       'TEAM_ABBREVIATION_B', 'TEAM_CITY_B', 'STARTERS_BENCH_B', 'MIN_B'])
    starters = starters.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'STARTERS_BENCH_A', 'MIN_A','TEAM_ID_B', 'TEAM_NAME_B',
       'TEAM_ABBREVIATION_B', 'TEAM_CITY_B', 'STARTERS_BENCH_B', 'MIN_B'])
    for columnName in bench:
        if((columnName == 'GAME_ID') | (columnName == 'TEAM_ID_A')):
            continue
        new_col_name = columnName + '_bench'
        bench[new_col_name] = bench[columnName]
        bench = bench.drop(columns = [columnName])
    for columnName in starters:
        if((columnName == 'GAME_ID') | (columnName == 'TEAM_ID_A')):
            continue
        new_col_name = columnName + '_starters'
        starters[new_col_name] = starters[columnName]
        starters = starters.drop(columns = [columnName])
        
    merged = pd.merge(starters,bench,left_on = 'GAME_ID',right_on = 'GAME_ID')
    merged = merged.drop_duplicates(subset = ['GAME_ID','TEAM_ID_A_x'])
    merged = merged.drop(columns = ['TEAM_ID_A_y'])
    merged = merged.rename(columns={"TEAM_ID_A_x": "TEAM_ID_A"})
    return(merged)
    

In [11]:
def four_factors_clean(four):
    four = four.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'MIN_A','TEAM_ABBREVIATION_B',
       'TEAM_CITY_B', 'MIN_B','OREB_PCT_A','TM_TOV_PCT_A','EFG_PCT_A'])
    return four

In [12]:
def usage_clean(usage):
    return usage
    

In [13]:
def misc_clean(misc):
    misc = misc.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'MIN_A'])
    return misc

In [14]:
def get_minutes(matchupmisc):
    matchupmisc = matchupmisc.reset_index(drop=True)
    matchupmisc['minutes'] = (matchupmisc['MIN_A'].str[0:3].astype(int))/5
    minutes =  matchupmisc['minutes']
    return minutes
    
    

In [15]:
def per_minute_stats(frame, minutes):
    frame = frame.reset_index(drop = True)
    features = frame.drop(columns = ['GAME_ID'])
    game_id = frame['GAME_ID'].reset_index(drop = True)
    team_id = frame['TEAM_ID_A'].reset_index(drop=True)
    for columnName in features:
        if(columnName[-1] == 'B'):
            features = features.drop(columns = [columnName])
    for columnName in features:
        if(columnName[-1] == 'A'):
            features = features.rename(columns={columnName: columnName[0:-2]})
    features['minutes'] = minutes
    features = features.groupby('TEAM_ID').cumsum()
    for columnName in features:
        if((columnName == 'minutes') | (columnName == 'TEAM_ID')):
            continue
        new_col_name = columnName + '_per_minute'
        features[new_col_name] = features[columnName] / features['minutes']
        features = features.drop(columns = [columnName])
    features = features.reset_index(drop= True)
    features= features.drop(columns = ['minutes'])
    features['TEAM_ID'] = team_id
    features['GAME_ID'] = game_id
    return combine_team_games(features, keep_method = None)
    
        
    

In [16]:
def cum_stats(frame):
    frame = frame.reset_index(drop = True)
    features = frame.drop(columns = ['GAME_ID'])
    game_id = frame['GAME_ID'].reset_index(drop = True)
    team_id = frame['TEAM_ID_A'].reset_index(drop=True)
    for columnName in features:
        if(columnName[-1] == 'B'):
            features = features.drop(columns = [columnName])
    for columnName in features:
        if(columnName[-1] == 'A'):
            features = features.rename(columns={columnName: columnName[0:-2]})
    features = features.reset_index(drop = True)
    for columnName in features:
        if(columnName == 'TEAM_ID'):
            continue
        new_col_name = columnName + '_median'
        features[new_col_name] = features.groupby('TEAM_ID')[columnName].apply(lambda x: x.shift().expanding().median())
        features = features.drop(columns = [columnName])
    features = features.reset_index(drop= True)
    features['TEAM_ID'] = team_id
    features['GAME_ID'] = game_id
    features = features.reset_index(drop = True)
    return combine_team_games(features, keep_method = None)

In [17]:
def get_features(merged):
    for columnName in merged:
        if((columnName == 'GAME_ID') | (columnName[0:7] == 'TEAM_ID')):
            merged = merged.drop(columns = [columnName])
    return merged
    

In [18]:
def reg_impute(reg_cleaned, minutes):
    per_minute = reg_cleaned[['TEAM_ID_A', 'GAME_ID', 'PTS_A', 'FGM_A', 'FGA_A',
                      'FG3M_A','FG3A_A','FTM_A', 'FTA_A','OREB_A', 'DREB_A',
       'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PLUS_MINUS_A']]
    
    cum = reg_cleaned[['TEAM_ID_A', 'GAME_ID','FG_PCT_A','FG3_PCT_A','FT_PCT_A']]
    
    per_minute = per_minute_stats(per_minute, minutes)
    cum = cum_stats(cum)
    merged = pd.merge(cum,per_minute,left_on = ['GAME_ID','TEAM_ID_A'],right_on=['GAME_ID','TEAM_ID_A'])
    #merged = merged.drop_duplicates(subset = ['TEAM_ID_A_x','GAME_ID','TEAM_ID_B_x'])
    merged = merged.rename(columns = {'TEAM_ID_A_x':'TEAM_ID'})
    for columnName in merged:
        if(columnName[-1] == 'B'):
            merged = merged.drop(columns = [columnName])
    for columnName in merged:
        if(columnName[-1] == 'A'):
            merged = merged.rename(columns={columnName: columnName[0:-2]})
    merged = merged.drop(columns = ['TEAM_ID_B_x','TEAM_ID_B_y'])
    merged = combine_team_games(merged,keep_method = None)
    #return merged
    merged['GAME_ID'] = merged['GAME_ID'].str[2:].astype(int)
    return merged


In [19]:
def get_62(frame):
    return frame.iloc[600:,:]

In [20]:
def four_impute(four_cleaned):
    cum = cum_stats(four_cleaned)
    cum['GAME_ID'] = cum['GAME_ID'].str[2:].astype(int)
    return cum

In [21]:
def advanced_impute(advanced_cleaned):
    cum = cum_stats(advanced_cleaned)
    return cum

In [22]:
def merger(reg_imputed, advanced_imputed, four_imputed):
    merged = pd.merge(reg_imputed,advanced_imputed, left_on = ['GAME_ID','TEAM_ID_A'],right_on = ['GAME_ID','TEAM_ID_A'])
    merged = pd.merge(merged,four_imputed, left_on = ['GAME_ID','TEAM_ID_A'],right_on = ['GAME_ID','TEAM_ID_A'])
    return get_62(merged)

In [23]:
def dataloader(year,start_date,end_date):
    reg = pull_reg_season(year,start_date,end_date)
    misc = pull_stats_slow(boxscoremiscv2.BoxScoreMiscV2,1,reg)
    minutes = get_minutes(misc)
    advanced = pull_stats_slow(boxscoreadvancedv2.BoxScoreAdvancedV2,1,reg)
    four = pull_stats_slow(boxscorefourfactorsv2.BoxScoreFourFactorsV2,1,reg)
    merged = merger(reg_impute(reg_clean(reg), minutes),advanced_impute(advancedboxscore_clean(advanced)),
                    four_impute(four_factors_clean(four)))
    merged.to_csv(year + '.csv')
    return merged

In [521]:
merged = merger(reg_test,advanced_test,four_test)

In [519]:
four_test = four_impute(four_factors_clean(four))

In [509]:
advanced_test.columns

Index(['TEAM_ID_A', 'E_OFF_RATING_median_A', 'OFF_RATING_median_A',
       'E_DEF_RATING_median_A', 'DEF_RATING_median_A', 'E_NET_RATING_median_A',
       'NET_RATING_median_A', 'AST_PCT_median_A', 'AST_TOV_median_A',
       'AST_RATIO_median_A', 'OREB_PCT_median_A', 'DREB_PCT_median_A',
       'REB_PCT_median_A', 'E_TM_TOV_PCT_median_A', 'TM_TOV_PCT_median_A',
       'EFG_PCT_median_A', 'TS_PCT_median_A', 'USG_PCT_median_A',
       'E_USG_PCT_median_A', 'E_PACE_median_A', 'PACE_median_A',
       'PACE_PER40_median_A', 'POSS_median_A', 'PIE_median_A', 'GAME_ID',
       'TEAM_ID_B', 'E_OFF_RATING_median_B', 'OFF_RATING_median_B',
       'E_DEF_RATING_median_B', 'DEF_RATING_median_B', 'E_NET_RATING_median_B',
       'NET_RATING_median_B', 'AST_PCT_median_B', 'AST_TOV_median_B',
       'AST_RATIO_median_B', 'OREB_PCT_median_B', 'DREB_PCT_median_B',
       'REB_PCT_median_B', 'E_TM_TOV_PCT_median_B', 'TM_TOV_PCT_median_B',
       'EFG_PCT_median_B', 'TS_PCT_median_B', 'USG_PCT_median_B',
 

In [520]:
four_test.columns

Index(['TEAM_ID_A', 'FTA_RATE_median_A', 'OPP_EFG_PCT_median_A',
       'OPP_FTA_RATE_median_A', 'OPP_TOV_PCT_median_A',
       'OPP_OREB_PCT_median_A', 'GAME_ID', 'TEAM_ID_B', 'FTA_RATE_median_B',
       'OPP_EFG_PCT_median_B', 'OPP_FTA_RATE_median_B', 'OPP_TOV_PCT_median_B',
       'OPP_OREB_PCT_median_B'],
      dtype='object')

In [496]:
seventeen = pd.read_csv('archive/2017-18/vegas.txt',sep=",")

In [497]:
seventeen = seventeen[['Pinnacle_Line_OU',
       'Pinnacle_Odds_OU','GameId','Total']]

In [522]:
test = pd.merge(merged,seventeen,left_on= 'GAME_ID',right_on = 'GameId',how='inner')

In [523]:
test = test.drop_duplicates(subset = ['GAME_ID','TEAM_ID_A'])

In [524]:
test.columns

Index(['TEAM_ID_A', 'FG_PCT_median_A', 'FG3_PCT_median_A', 'FT_PCT_median_A',
       'GAME_ID', 'PTS_per_minute_A', 'FGM_per_minute_A', 'FGA_per_minute_A',
       'FG3M_per_minute_A', 'FG3A_per_minute_A',
       ...
       'TEAM_ID_B', 'FTA_RATE_median_B', 'OPP_EFG_PCT_median_B',
       'OPP_FTA_RATE_median_B', 'OPP_TOV_PCT_median_B',
       'OPP_OREB_PCT_median_B', 'Pinnacle_Line_OU', 'Pinnacle_Odds_OU',
       'GameId', 'Total'],
      dtype='object', length=103)

In [535]:
test_features = test.drop(columns = ['TEAM_ID_A','GAME_ID','TEAM_ID_B','Pinnacle_Line_OU', 'Pinnacle_Odds_OU'
                                    ,'GameId','TEAM_ID_B_x','TEAM_ID_B_y'])

In [530]:
test_features = test.drop(columns = ['TEAM_ID_B_y'])

In [538]:
target = test_features['Total']

In [540]:
test_features = test_features.drop(columns = ['Total'])

In [541]:
target.to_csv('target_test.csv')
test_features.to_csv('features_test.csv')

In [555]:
merged2018 = dataloader('2018','2018-10-15','2019-04-11')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [24]:
reg1819 = pull_reg_season('2018','2018-10-15','2019-04-11')

In [25]:
reg1819

,SEASON_ID_A,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE_A,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
1,22018,1610612750,MIN,Minnesota Timberwolves,0021801228,2019-04-10,MIN @ DEN,L,240,95,...,0.647,12,41,53,23,6,4,12,12,4.0
2,22018,1610612743,DEN,Denver Nuggets,0021801228,2019-04-10,DEN vs. MIN,W,241,99,...,0.667,7,34,41,24,6,0,10,22,-4.0
5,22018,1610612759,SAS,San Antonio Spurs,0021801227,2019-04-10,SAS vs. DAL,W,242,105,...,0.750,7,35,42,27,5,4,7,19,-11.0
6,22018,1610612742,DAL,Dallas Mavericks,0021801227,2019-04-10,DAL @ SAS,L,242,94,...,0.833,8,45,53,22,6,2,10,14,11.0
9,22018,1610612746,LAC,LA Clippers,0021801229,2019-04-10,LAC vs. UTA,W,264,143,...,0.879,17,40,57,31,8,11,17,24,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,22018,1610612749,MIL,Milwaukee Bucks,0021800003,2018-10-17,MIL @ CHA,W,240,113,...,0.636,9,32,41,21,8,9,11,19,-1.0
4913,22018,1610612760,OKC,Oklahoma City Thunder,0021800002,2018-10-16,OKC @ GSW,L,240,100,...,0.944,17,41,58,28,7,7,21,29,8.0
4914,22018,1610612744,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,...,0.649,16,29,45,21,12,6,14,21,-8.0
4917,22018,1610612755,PHI,Philadelphia 76ers,0021800001,2018-10-16,PHI @ BOS,L,239,87,...,0.714,12,43,55,21,7,5,14,20,18.0


In [28]:
misc18 = pull_stats_super_slow(boxscoremiscv2.BoxScoreMiscV2,1,reg1819)
misc18.to_csv('../../misc18.csv')

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [30]:
advanced18 = pull_stats_super_slow(boxscoreadvancedv2.BoxScoreAdvancedV2,1,reg1819)
advanced18.to_csv('../../advanced18.csv')

ConnectionError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/boxscoreadvancedv2?EndPeriod=0&EndRange=0&GameID=0021800820&RangeType=0&StartPeriod=0&StartRange=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x116d8f130>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
four18 = pull_stats_slow(boxscorefourfactorsv2.BoxScoreFourFactorsV2,1,reg1819)
four18.to_csv('../../four18.csv')

In [552]:
total = seventeen['Total'][600:]

In [553]:
OU = seventeen['Pinnacle_Line_OU'][600:]

In [554]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(OU, total))

17.946650329922743

In [ ]:
from pyearth import Earth
m5 = Earth(max_terms=500, max_degree=4) # note, terms in brackets are the hyperparameters
m5.fit(X_train,y_train)
print(m5.summary())